In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

In [2]:
raw = pd.read_csv('tr_eikon_eod_data.csv', index_col = 0, parse_dates = True)
data = raw[['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']].dropna()
data.plot()

FileNotFoundError: [Errno 2] No such file or directory: 'tr_eikon_eod_data.csv'

In [ ]:
# split the dataset into 50 (training) / 50 (testing)
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data,
                                         test_size = 0.5,
                                         shuffle = True,
                                         random_state = 0) 

# Calculate returns for benchmark case

In [ ]:
log_ret_train = (np.log(data_train / data_train.shift(1))).dropna()
log_ret_train

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N
Date,,,,,
2018-05-14,0.914680,0.951378,0.796284,1.492256,0.388553
2013-10-02,-0.989644,-1.061269,-0.875032,-1.608807,-0.429973
2014-03-28,0.092230,0.172347,0.112892,0.053990,0.022620
2016-05-24,0.244120,0.246976,0.192548,0.733159,-0.029450
2015-02-17,0.266755,-0.168730,0.111827,-0.628990,0.187133
...,...,...,...,...,...
2014-02-11,0.411379,0.273393,0.123136,0.681250,-0.024991
2016-11-16,0.362241,0.472858,0.353318,0.724318,0.226896
2013-01-15,-0.460257,-0.784910,-0.465193,-1.009948,-0.419502


In [ ]:
log_ret_train['Returns'] = log_ret_train['AAPL.O'] + log_ret_train['MSFT.O'] + log_ret_train['INTC.O'] + log_ret_train['AMZN.O']

log_ret_train

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,Returns
Date,,,,,,
2018-05-14,0.914680,0.951378,0.796284,1.492256,0.388553,4.154598
2013-10-02,-0.989644,-1.061269,-0.875032,-1.608807,-0.429973,-4.534752
2014-03-28,0.092230,0.172347,0.112892,0.053990,0.022620,0.431459
2016-05-24,0.244120,0.246976,0.192548,0.733159,-0.029450,1.416803
2015-02-17,0.266755,-0.168730,0.111827,-0.628990,0.187133,-0.419138
...,...,...,...,...,...,...
2014-02-11,0.411379,0.273393,0.123136,0.681250,-0.024991,1.489157
2016-11-16,0.362241,0.472858,0.353318,0.724318,0.226896,1.912735
2013-01-15,-0.460257,-0.784910,-0.465193,-1.009948,-0.419502,-2.720307


# AAPL.O stepwise regression

In [7]:
AAPLO_train = pd.DataFrame(data_train['AAPL.O'])
AAPLO_test = pd.DataFrame(data_test['AAPL.O'])

In [8]:
# Prepare train data set 
AAPLO_train['log_ret'] = np.log(AAPLO_train['AAPL.O'] / AAPLO_train['AAPL.O'].shift(1))

for i in range(1,6):
    s = 'lag'+ str(i)
    AAPLO_train[s] = AAPLO_train['log_ret'].shift(i)
AAPLO_train = AAPLO_train.dropna()
AAPLO_train

,AAPL.O,log_ret,lag1,lag2,lag3,lag4,lag5
Date,,,,,,,
2010-03-16,32.064254,-1.382959,0.266755,0.244120,0.092230,-0.989644,0.914680
2012-02-24,74.629925,0.844800,-1.382959,0.266755,0.244120,0.092230,-0.989644
2010-06-11,36.215678,-0.723049,0.844800,-1.382959,0.266755,0.244120,0.092230
2012-01-11,60.364225,0.510905,-0.723049,0.844800,-1.382959,0.266755,0.244120
2010-03-09,31.859968,-0.639046,0.510905,-0.723049,0.844800,-1.382959,0.266755
...,...,...,...,...,...,...,...
2014-02-11,76.565638,0.411379,-0.876759,-0.161690,0.461229,-0.223840,0.429866
2016-11-16,109.990000,0.362241,0.411379,-0.876759,-0.161690,0.461229,-0.223840
2013-01-15,69.417073,-0.460257,0.362241,0.411379,-0.876759,-0.161690,0.461229


In [9]:
# Stepwise regression select features

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

LR_model = LinearRegression()
SFS_model = SequentialFeatureSelector(LR_model,
                                      scoring = 'r2')

X_train = AAPLO_train.iloc[:,2:]
y_train = AAPLO_train.iloc[:,1]

sel_X_bool = SFS_model.fit(X_train, y_train).get_support()
sel_X_train = X_train.loc[:, sel_X_bool]
sel_X_train

,lag1,lag2
Date,,
2010-03-16,0.266755,0.244120
2012-02-24,-1.382959,0.266755
2010-06-11,0.844800,-1.382959
2012-01-11,-0.723049,0.844800
2010-03-09,0.510905,-0.723049
...,...,...
2014-02-11,-0.876759,-0.161690
2016-11-16,0.411379,-0.876759
2013-01-15,0.362241,0.411379


#### Prediciton on training data

In [10]:
# perform linear regression on the selected variables to predict training data
AAPLO_train['Predict_Step'] = LR_model.fit(sel_X_train, y_train).predict(sel_X_train)
AAPLO_train

,AAPL.O,log_ret,lag1,lag2,lag3,lag4,lag5,Predict_Step
Date,,,,,,,,
2010-03-16,32.064254,-1.382959,0.266755,0.244120,0.092230,-0.989644,0.914680,-0.252010
2012-02-24,74.629925,0.844800,-1.382959,0.266755,0.244120,0.092230,-0.989644,0.749425
2010-06-11,36.215678,-0.723049,0.844800,-1.382959,0.266755,0.244120,0.092230,-0.017377
2012-01-11,60.364225,0.510905,-0.723049,0.844800,-1.382959,0.266755,0.244120,0.136526
2010-03-09,31.859968,-0.639046,0.510905,-0.723049,0.844800,-1.382959,0.266755,-0.051675
...,...,...,...,...,...,...,...,...
2014-02-11,76.565638,0.411379,-0.876759,-0.161690,0.461229,-0.223840,0.429866,0.594569
2016-11-16,109.990000,0.362241,0.411379,-0.876759,-0.161690,0.461229,-0.223840,0.064820
2013-01-15,69.417073,-0.460257,0.362241,0.411379,-0.876759,-0.161690,0.461229,-0.370932


In [11]:
AAPLO_train['dir'] = np.sign(AAPLO_train['Predict_Step'])
AAPLO_train['Predict_dir'] = np.sign(AAPLO_train['Predict_Step'])
AAPLO_train

,AAPL.O,log_ret,lag1,lag2,lag3,lag4,lag5,Predict_Step,dir,Predict_dir
Date,,,,,,,,,,
2010-03-16,32.064254,-1.382959,0.266755,0.244120,0.092230,-0.989644,0.914680,-0.252010,-1.0,-1.0
2012-02-24,74.629925,0.844800,-1.382959,0.266755,0.244120,0.092230,-0.989644,0.749425,1.0,1.0
2010-06-11,36.215678,-0.723049,0.844800,-1.382959,0.266755,0.244120,0.092230,-0.017377,-1.0,-1.0
2012-01-11,60.364225,0.510905,-0.723049,0.844800,-1.382959,0.266755,0.244120,0.136526,1.0,1.0
2010-03-09,31.859968,-0.639046,0.510905,-0.723049,0.844800,-1.382959,0.266755,-0.051675,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
2014-02-11,76.565638,0.411379,-0.876759,-0.161690,0.461229,-0.223840,0.429866,0.594569,1.0,1.0
2016-11-16,109.990000,0.362241,0.411379,-0.876759,-0.161690,0.461229,-0.223840,0.064820,1.0,1.0
2013-01-15,69.417073,-0.460257,0.362241,0.411379,-0.876759,-0.161690,0.461229,-0.370932,-1.0,-1.0


In [12]:
# number of trades in total
(AAPLO_train['Predict_dir'].diff().dropna() != 0).sum()

571

In [13]:
# number of false prediction
(AAPLO_train['Predict_dir'] != AAPLO_train['dir']).sum()

0

In [14]:
# Calculate return from strategy
AAPLO_train['Returns_Step'] = AAPLO_train['Predict_dir'] * AAPLO_train['log_ret']

np.exp(np.sum(AAPLO_train[['log_ret', 'Returns_Step']]))

log_ret          8.162403e-01
Returns_Step    2.113536e+144
dtype: float64

#### Testings

In [16]:
# Prepare test data
AAPLO_test['log_ret'] = np.log(AAPLO_test['AAPL.O'] / AAPLO_test['AAPL.O'].shift(1))

for i in range(1,6):
    s = 'lag'+ str(i)
    AAPLO_test[s] = AAPLO_test['log_ret'].shift(i)
AAPLO_test = AAPLO_test.dropna()

X_test = AAPLO_test.iloc[:,2:]
sel_X_test = X_test.loc[:, sel_X_bool]
sel_X_test

,lag1,lag2
Date,,
2015-04-27,-0.127975,-0.189779
2012-09-17,0.322325,-0.127975
2011-12-07,-0.282858,0.322325
2017-10-23,-0.586957,-0.282858
2012-10-01,1.033046,-0.586957
...,...,...
2015-08-24,0.784885,-0.164149
2014-07-28,-0.506413,0.784885
2017-07-07,-0.040572,-0.506413


In [17]:
# perform linear regression on the selected variables to predict test data
AAPLO_test['Predict_Step'] = LR_model.fit(sel_X_train, y_train).predict(sel_X_test)
AAPLO_test

,AAPL.O,log_ret,lag1,lag2,lag3,lag4,lag5,Predict_Step
Date,,,,,,,,
2015-04-27,132.650000,0.322325,-0.127975,-0.189779,-0.134270,0.441259,-0.501954,0.146473
2012-09-17,99.968614,-0.282858,0.322325,-0.127975,-0.189779,-0.134270,0.441259,-0.151459
2011-12-07,55.584230,-0.586957,-0.282858,0.322325,-0.127975,-0.189779,-0.134270,0.056071
2017-10-23,156.170000,1.033046,-0.586957,-0.282858,0.322325,-0.127975,-0.189779,0.461028
2012-10-01,94.198477,-0.505541,1.033046,-0.586957,-0.282858,0.322325,-0.127975,-0.420438
...,...,...,...,...,...,...,...,...
2015-08-24,103.120000,-0.506413,0.784885,-0.164149,0.527469,-0.408181,-0.203986,-0.421463
2014-07-28,99.020000,-0.040572,-0.506413,0.784885,-0.164149,0.527469,-0.408181,0.025612
2017-07-07,144.180000,0.375741,-0.040572,-0.506413,0.784885,-0.164149,0.527469,0.207485


In [18]:
AAPLO_test['dir'] = np.sign(AAPLO_test['Predict_Step'])
AAPLO_test

,AAPL.O,log_ret,lag1,lag2,lag3,lag4,lag5,Predict_Step,dir
Date,,,,,,,,,
2015-04-27,132.650000,0.322325,-0.127975,-0.189779,-0.134270,0.441259,-0.501954,0.146473,1.0
2012-09-17,99.968614,-0.282858,0.322325,-0.127975,-0.189779,-0.134270,0.441259,-0.151459,-1.0
2011-12-07,55.584230,-0.586957,-0.282858,0.322325,-0.127975,-0.189779,-0.134270,0.056071,1.0
2017-10-23,156.170000,1.033046,-0.586957,-0.282858,0.322325,-0.127975,-0.189779,0.461028,1.0
2012-10-01,94.198477,-0.505541,1.033046,-0.586957,-0.282858,0.322325,-0.127975,-0.420438,-1.0
...,...,...,...,...,...,...,...,...,...
2015-08-24,103.120000,-0.506413,0.784885,-0.164149,0.527469,-0.408181,-0.203986,-0.421463,-1.0
2014-07-28,99.020000,-0.040572,-0.506413,0.784885,-0.164149,0.527469,-0.408181,0.025612,1.0
2017-07-07,144.180000,0.375741,-0.040572,-0.506413,0.784885,-0.164149,0.527469,0.207485,1.0


In [19]:
AAPLO_test['Predict_dir'] = np.sign(AAPLO_test['Predict_Step'])
AAPLO_test

,AAPL.O,log_ret,lag1,lag2,lag3,lag4,lag5,Predict_Step,dir,Predict_dir
Date,,,,,,,,,,
2015-04-27,132.650000,0.322325,-0.127975,-0.189779,-0.134270,0.441259,-0.501954,0.146473,1.0,1.0
2012-09-17,99.968614,-0.282858,0.322325,-0.127975,-0.189779,-0.134270,0.441259,-0.151459,-1.0,-1.0
2011-12-07,55.584230,-0.586957,-0.282858,0.322325,-0.127975,-0.189779,-0.134270,0.056071,1.0,1.0
2017-10-23,156.170000,1.033046,-0.586957,-0.282858,0.322325,-0.127975,-0.189779,0.461028,1.0,1.0
2012-10-01,94.198477,-0.505541,1.033046,-0.586957,-0.282858,0.322325,-0.127975,-0.420438,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
2015-08-24,103.120000,-0.506413,0.784885,-0.164149,0.527469,-0.408181,-0.203986,-0.421463,-1.0,-1.0
2014-07-28,99.020000,-0.040572,-0.506413,0.784885,-0.164149,0.527469,-0.408181,0.025612,1.0,1.0
2017-07-07,144.180000,0.375741,-0.040572,-0.506413,0.784885,-0.164149,0.527469,0.207485,1.0,1.0


In [20]:
# number of trades in total
(AAPLO_test['Predict_dir'].diff().dropna() != 0).sum()

569

In [21]:
# number of false prediction
(AAPLO_test['Predict_dir'] != AAPLO_test['dir']).sum()

0

In [22]:
# Calculate return from strategy
AAPLO_test['Returns_Step'] = AAPLO_test['Predict_dir'] * AAPLO_test['log_ret']

np.exp(np.sum(AAPLO_test[['log_ret', 'Returns_Step']]))

log_ret          9.819969e-01
Returns_Step    5.434996e+135
dtype: float64